In [ ]:
columns_to_drop = ['Unnamed: 0','text', 'line_number', 'total_lines']
df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
# Save the preprocessed data
df.to_csv('preprocessed_dataset.csv', index=False)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv('preprocessed_dataset.csv')

# Split the dataset into features (X) and target (y)
X = df['processed_text']
y = df['target']

In [3]:
df['processed_text'].fillna('', inplace=True)
# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_tfidf = vectorizer.fit_transform(X)


In [4]:
# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

from sklearn.model_selection import train_test_split

# Take only 10% of the data
subset_size = int(0.5 * X_tfidf.shape[0])
X_subset = X_tfidf[:subset_size]
y_subset = y[:subset_size]

# Split the subset data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.2, random_state=42)

In [5]:
# Train Multinomial Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_predictions = nb_model.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_predictions)
print("Multinomial Naive Bayes Accuracy:", nb_accuracy)

Multinomial Naive Bayes Accuracy: 0.6911694230195401


In [6]:
# Train Logistic Regression
lr_model = LogisticRegression(max_iter=100)
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)
lr_accuracy = accuracy_score(y_test, lr_predictions)
print("Logistic Regression Accuracy:", lr_accuracy)


C:\Users\asus\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.76708290759813


In [ ]:
# Train Support Vector Machine
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print("Support Vector Machine Accuracy:", svm_accuracy)

In [15]:
# Split the TF-IDF matrix into training and validation sets
#X_train_nn, X_val_nn, y_train_nn, y_val_nn = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

from sklearn.model_selection import train_test_split

# Take only 10% of the training data
subset_size_train = int(0.3 * X_train.shape[0])
X_train_subset = X_train[:subset_size_train]
y_train_subset = y_train[:subset_size_train]

# Split the subset training data into training and validation sets
X_train_nn, X_val_nn, y_train_nn, y_val_nn = train_test_split(X_train_subset, y_train_subset, test_size=0.2, random_state=42)


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# Define a revised neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),  # Adding dropout for regularization
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),  # Adding dropout for regularization
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dense(len(df['target'].unique()), activation='softmax')
])


In [17]:
from tensorflow.keras.optimizers import Adam
# Compile the model with Adam optimizer and a reduced learning rate
model.compile(optimizer=Adam(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

C:\Users\asus\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [18]:
# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [19]:
from sklearn.preprocessing import LabelEncoder

# Encode target labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_nn)
y_val_encoded = label_encoder.transform(y_val_nn)


In [20]:
import numpy as np
import scipy

# Sort the indices of the sparse matrices
def sort_sparse_indices(X):
    if not isinstance(X, (scipy.sparse.csr_matrix, scipy.sparse.csc_matrix)):
        raise ValueError("X must be a scipy sparse matrix.")

    if not X.has_sorted_indices:
        X.sort_indices()

In [21]:
# Sort the indices of the training and validation sets
sort_sparse_indices(X_train_nn)
sort_sparse_indices(X_val_nn)

In [22]:
# Train the model
history = model.fit(X_train_nn, y_train_encoded,
                    epochs=15,
                    batch_size=64,
                    validation_data=(X_val_nn, y_val_encoded),
                    callbacks=[early_stopping])

Epoch 1/15


C:\Users\asus\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_1/dense_4/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_1/dense_4/embedding_lookup_sparse/Reshape:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_1/dense_4/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


3318/3318 [==============================] - 469s 141ms/step - loss: 0.8473 - accuracy: 0.6810 - val_loss: 0.6844 - val_accuracy: 0.7446
Epoch 2/15
3318/3318 [==============================] - 449s 135ms/step - loss: 0.6505 - accuracy: 0.7638 - val_loss: 0.6652 - val_accuracy: 0.7517
Epoch 3/15
3318/3318 [==============================] - 454s 137ms/step - loss: 0.5751 - accuracy: 0.7943 - val_loss: 0.6617 - val_accuracy: 0.7559
Epoch 4/15
3318/3318 [==============================] - 450s 135ms/step - loss: 0.5116 - accuracy: 0.8205 - val_loss: 0.6735 - val_accuracy: 0.7540
Epoch 5/15
3318/3318 [==============================] - 459s 138ms/step - loss: 0.4609 - accuracy: 0.8400 - val_loss: 0.6901 - val_accuracy: 0.7557
Epoch 6/15
3318/3318 [==============================] - 450s 135ms/step - loss: 0.4170 - accuracy: 0.8561 - val_loss: 0.7125 - val_accuracy: 0.7512


In [24]:
# Assuming 'model' is your trained model object
model.save("my_model.h5")


In [23]:
import tensorflow as tf

# Convert X_test to a dense tensor for evaluation
X_test_dense = X_test.toarray()  # Assuming X_test is a sparse matrix

# Create a TensorFlow sparse tensor with correctly ordered indices
X_test_sparse = tf.sparse.reorder(tf.sparse.from_dense(X_test_dense))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_sparse, y_test_encoded)
print(f"Neural Network Accuracy: {accuracy:.2f}")


MemoryError: Unable to allocate 215. GiB for an array with shape (221186, 130591) and data type float64